# Using `numba.jit` to speedup the computation of the Euclidean distance matrix 

In this notebook we implement a function to compute the Euclidean distance matrix using Numba's *just-in-time* compilation decorator. We compare it with the NumPy function we wrote before.

In [1]:
import numpy as np
import numba

In [2]:
@numba.jit(nopython=True)
def euclidean_numba(x, y):
    """Implementation with numba."""

    num_samples, num_feat = x.shape
    dist_matrix = np.zeros((num_samples, num_samples))
    for i in range(num_samples):
        for j in range(num_samples):
            r = 0.0
            for k in range(num_feat):
                r += (x[i][k] - y[j][k])**2
            dist_matrix[i][j] = r

    return dist_matrix

Let's include here our numpy implementation for comparison.

In [3]:
def euclidean_numpy(x, y):
    """Euclidean square distance matrix.
    
    Inputs:
    x: (N, m) numpy array
    y: (N, m) numpy array
    
    Ouput:
    (N, N) Euclidean square distance matrix:
    r_ij = (x_ij - y_ij)^2
    """

    x2 = np.einsum('ij,ij->i', x, x)[:, np.newaxis]
    y2 = np.einsum('ij,ij->i', y, y)[:, np.newaxis].T

    xy = np.dot(x, y.T)

    return np.abs(x2 + y2 - 2. * xy)

### Exercise 1
Before runing the different functions, could you say which of the two numba implementations would be faster?

In [4]:
# Let's check that they all give the same result
a = 10. * np.random.random([100, 10])

print(np.abs(euclidean_numpy(a, a) - euclidean_numba(a, a)).max())

3.410605131648481e-13


Our Numba implementations can be faster than the NumPy one for a list of small vectors. However, with larger vectors, the NumPy implementation is faster:

In [5]:
nsamples = 100
nfeat = 3

x = 10. * np.random.random([nsamples, nfeat])

%timeit euclidean_numpy(x, x)
%timeit euclidean_numba(x, x)

88.2 µs ± 675 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
70 µs ± 640 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
nsamples = 100
nfeat = 50

x = 10. * np.random.random([nsamples, nfeat])

%timeit euclidean_numpy(x, x)
%timeit euclidean_numba(x, x)

94.5 µs ± 95.8 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
845 µs ± 1.14 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In a more realistic case, our NumPy implementation is much faster:

In [7]:
nsamples = 5000
nfeat = 50

x = 10. * np.random.random([nsamples, nfeat])

%timeit euclidean_numpy(x, x)
%timeit euclidean_numba(x, x)

577 ms ± 6.28 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.19 s ± 10.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
